In [1]:
import pandas as pd
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

In [2]:
from src.utils import load_credentials
from src.upload_data import upload_data
from src.clean_data import get_df
from src.clean_data import join_all

In [3]:
credentials = load_credentials("blob_storage")

# Create and show containers

In [4]:
blob_service_client = BlobServiceClient\
    .from_connection_string(credentials['conn_string'])

In [10]:
# Create a unique name for the container
container_name = "raw-data"

try:
    container_client = blob_service_client.create_container(container_name)
    properties = container_client.get_container_properties()
except:
    print("Container already exists.")

Container already exists.


In [11]:
all_containers = blob_service_client.list_containers()
for c in all_containers:
    print(c.name)

raw-data


# Upload data

In [16]:
# Start client
blob_service_client = BlobServiceClient.from_connection_string(credentials['conn_string'])

In [17]:
# Upload all files
container_name = "raw-data"
blob_clients = upload_data(blob_service_client, container=container_name)

Writing: .\data\401kRevenue.txt
Writing: .\data\401kRevenueDetail.txt
Writing: .\data\ASORevenue.txt
Writing: .\data\ASORevenueDrivers.txt
Writing: .\data\BlendedProductRevenue.txt
Writing: .\data\BlendedProductRevenueDrivers.txt
Writing: .\data\IFHC.txt
Writing: .\data\InsuranceRevenue.txt
Writing: .\data\InsuranceRevenueDetail.txt
Writing: .\data\InternationalRevenue.txt
Writing: .\data\InternationalRevenueStats.txt
Writing: .\data\OnlineRevenue.txt
Writing: .\data\OnlineRevenueDetail.txt
Writing: .\data\OtherMgmtRevenue.txt
Writing: .\data\OtherMgmtRevenueDetail.txt
Writing: .\data\PayrollSurePayrollASOInternationalHighLevelRevenue.txt
Writing: .\data\PEORevenue.txt
Writing: .\data\PEORevenueDetail.txt
Writing: .\data\PEORevenueDetailDrivers.txt
Writing: .\data\SurePayollRevenue.txt
Writing: .\data\SurePayollRevenueDrivers.txt
Writing: .\data\W2DelOtherRevenue.txt
Writing: .\data\W2DelOtherRevenueDrivers.txt
Writing: .\data\oldfiles\401kRevenue.txt
Writing: .\data\oldfiles\401kReven

# Clean Data

In [4]:
# Start client
container_name = "raw-data"
blob_service_client = BlobServiceClient.from_connection_string(credentials['conn_string'])
# container_client = blob_service_client.get_container_client(container_name)

In [5]:
def get_blob_list(client, container="raw-data"):
    """
    Get blobs in a container
    """
    container_client = client.get_container_client(container)
    blob_list = []
    for blob in container_client.list_blobs():
        file_name = blob.name
        blob_list.append(file_name)

    return blob_list

In [6]:
blob_list = get_blob_list(blob_service_client)

In [7]:
blob_list

['401kRevenue.txt',
 '401kRevenueDetail.txt',
 'ASORevenue.txt',
 'ASORevenueDrivers.txt',
 'BlendedProductRevenue.txt',
 'BlendedProductRevenueDrivers.txt',
 'IFHC.txt',
 'InsuranceRevenue.txt',
 'InsuranceRevenueDetail.txt',
 'InternationalRevenue.txt',
 'InternationalRevenueStats.txt',
 'OnlineRevenue.txt',
 'OnlineRevenueDetail.txt',
 'OtherMgmtRevenue.txt',
 'OtherMgmtRevenueDetail.txt',
 'PEORevenue.txt',
 'PEORevenueDetail.txt',
 'PEORevenueDetailDrivers.txt',
 'PayrollSurePayrollASOInternationalHighLevelRevenue.txt',
 'SurePayollRevenue.txt',
 'SurePayollRevenueDrivers.txt',
 'W2DelOtherRevenue.txt',
 'W2DelOtherRevenueDrivers.txt',
 'oldfiles/401kRevenue.txt',
 'oldfiles/401kRevenue.xlsx',
 'oldfiles/401kRevenueDetail.txt',
 'oldfiles/401kRevenueDetail.xlsx']

## 401kRevenue

In [25]:
df = get_df(blob_service_client, blob_list[0], container= container_name)\
    .reset_index(-1)\
    .rename(columns={'level_3':'period'})

#Correct month
df['period'] = df['period']\
    .replace({'\nJun': 1, '\nJul': 2, '\nAug': 3, '\nSep': 4, '\nOct': 5, '\nNov': 6, '\nDec': 7, '\nJan': 8,
             '\nFeb': 9, '\nMar': 10, '\nApr': 11, '\nMay': 12, 'YearTotal':0})\
    .astype('int')

In [29]:
df[df['period']==0]\
    .loc[("Forecast","6+6","FY22"),:].groupby(level=3).sum()/1000000

                                                0.000000
*fee revenue                                  189.683425
Average Asset Values - Calculated - RW      48171.515502
Average Assets/Clients - Calculated - RW        0.481760
Basis Points - RW                               0.000007
Basis Points - Solicitor - RW                  -0.000001
Basis Pts - Calc - RW                           0.000000
DIRECT BILL/MONEY MGMT REVENUE - RW           116.545445
Ending Asset Values - RW                    49850.985562
Ending Client Base - RW                         0.104507
Losses - RW                                    -0.013544
Maintenance Rate - Calculated - RW              0.000120
Sales - RW                                      0.022539
Set Up Rate - Calculated - RW                   0.000538
Total Service Revenue - RW                    306.228870
Name: (Forecast, 6+6, FY22), dtype: float64

In [27]:
df[df['period']==0]\
    .loc[("Forecast","6+6","FY22"),
         ("Total Activity","Total 401k Revenue", "Total Paychex", "Total Service Revenue - RW")]/1000000

306.22887

In [24]:
df[(df['period']<=3)&(df['period']>=1)]\
    .loc[("Forecast","6+6","FY22"),
         ("Total Activity", "Total 401k Revenue", "Total Paychex","Total Service Revenue - RW")]\
    .sum()/1000000

C:\Users\bruno.gonzalez\Miniconda3\envs\azml\lib\site-packages\pandas\core\indexing.py:873: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


72.761065

In [25]:
df[(df['period']<=6)&(df['period']>=4)]\
    .loc[("Forecast","6+6","FY22"),
         ("Total Activity", "Total 401k Revenue", "Total Paychex","Total Service Revenue - RW")]\
    .sum()/1000000

C:\Users\bruno.gonzalez\Miniconda3\envs\azml\lib\site-packages\pandas\core\indexing.py:873: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


77.886922

## Blended Product Revenue

In [21]:
df = get_df(blob_service_client, blob_list[4], container= container_name)\
    .reset_index(-1) \
    .rename(columns={'level_3':'period'})

    #Correct month
df['period'] = df['period']. \
    replace({'\nJun': 1, '\nJul': 2, '\nAug': 3, '\nSep': 4, '\nOct': 5, '\nNov': 6, '\nDec': 7, '\nJan': 8,
             '\nFeb': 9, '\nMar': 10, '\nApr': 11, '\nMay': 12, 'YearTotal':0}) \
    .astype('int')

In [22]:
df[df['period']==0]\
    .loc[("Forecast","6+6","FY22"),:].groupby(level=1).sum()/1000000

                                               0.000000
1-800 SUI SERVICES                            22.280636
Total Advantage Blended Products Revenue      27.996336
Total Flex Blended Products Revenue         1248.104272
Total Preview Blended Products Revenue        28.755500
Name: (Forecast, 6+6, FY22), dtype: float64

In [27]:
df["Total Activity", "Total Flex Blended Products Revenue", "Total","Total"] = df["Total Activity"].sum(axis=1)

In [28]:
df

period                      Total Activity  \
                                   Total Flex Blended Products Revenue   
                                                            Auburn RSC   
                                                 FEE REVENUE - NA - RW   
Actual   Final Revised FY15      0                         122970372.0   
                       FY15      1                           9624260.0   
                       FY15      2                          11784050.0   
                       FY15      3                           9592805.0   
                       FY15      4                           9327331.0   
...                            ...                                 ...   
Forecast 8+4           FY22      8                                 0.0   
                       FY22      9                                 0.0   
                       FY22     10                                 0.0   
                       FY22     11                                 0.0   
                       FY22     12                                 0.0   

                                                              \
                                                               
                                                               
                            FEE REVENUE - CONTRACT PLAN - RW   
Actual   Final Revised FY15                              0.0   
                       FY15                              0.0   
                       FY15                              0.0   
                       FY15                              0.0   
                       FY15                              0.0   
...                                                      ...   
Forecast 8+4           FY22                         259200.0   
                       FY22                         135806.0   
                       FY22                         182737.0   
                       FY22                         144785.0   
                       FY22                         138549.0   

                                                     \
                                                      
                                                      
                            OTHER REVENUE - NA - RW   
Actual   Final Revised FY15                     0.0   
                       FY15                     0.0   
                       FY15                     0.0   
                       FY15                     0.0   
                       FY15                     0.0   
...                                             ...   
Forecast 8+4           FY22                 10350.0   
                       FY22                     0.0   
                       FY22                     0.0   
                       FY22                     0.0   
                       FY22                     0.0   

                                                           \
                                                            
                                                            
                            MAINTENANCE REVENUE - NA - RW   
Actual   Final Revised FY15                           0.0   
                       FY15                           0.0   
                       FY15                           0.0   
                       FY15                           0.0   
                       FY15                           0.0   
...                                                   ...   
Forecast 8+4           FY22                     8642359.0   
                       FY22                     6881849.0   
                       FY22                     8242950.0   
                       FY22                     6611078.0   
                       FY22                     6671279.0   

                                                       \
                                                        
                                                        
                            SALES DISCOUNTS

In [29]:
df[df['period']==0]\
    .loc[("Forecast","6+6","FY22"),
         ("Total Activity", "Total Flex Blended Products Revenue", "Total","Total")]/1000000

1327.136744

In [30]:
df[(df['period']<=3)&(df['period']>=1)]\
    .loc[("Forecast","6+6","FY22"),
         ("Total Activity", "Total Flex Blended Products Revenue", "Total","Total")].sum()/1000000

C:\Users\bruno.gonzalez\Miniconda3\envs\azml\lib\site-packages\pandas\core\indexing.py:873: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


343.19276

## International Revenue

In [31]:
df = get_df(blob_service_client, blob_list[9], container= container_name)

In [32]:
df["Total Activity", "PR PROC", "Total","Total"] = df["Total Activity"].sum(axis=1)

In [33]:
df

period               Total Activity  \
                                                        PR PROC   
                                   3000 GERMANY GMBH OPERATIONS   
                                          FEE REVENUE - NA - RW   
Actual   Final Revised FY15      0                   4309124.40   
                       FY15      1                    369471.91   
                       FY15      2                    401379.94   
                       FY15      3                    365608.89   
                       FY15      4                    314304.69   
...                            ...                          ...   
Forecast 8+4           FY22      8                     24115.76   
                       FY22      9                      9530.09   
                       FY22     10                      8902.97   
                       FY22     11                      7912.84   
                       FY22     12                      7506.39   

                                                     \
                                                      
                                                      
                            OTHER REVENUE - NA - RW   
Actual   Final Revised FY15                    0.00   
                       FY15                    0.00   
                       FY15                    0.00   
                       FY15                    0.00   
                       FY15                    0.00   
...                                             ...   
Forecast 8+4           FY22               107274.28   
                       FY22                20647.05   
                       FY22                91301.90   
                       FY22                    0.00   
                       FY22                    0.00   

                                                           \
                                                            
                                                            
                            MAINTENANCE REVENUE - NA - RW   
Actual   Final Revised FY15                          0.00   
                       FY15                          0.00   
                       FY15                          0.00   
                       FY15                          0.00   
                       FY15                          0.00   
...                                                   ...   
Forecast 8+4           FY22                     547429.63   
                       FY22                     496008.45   
                       FY22                     531171.41   
                       FY22                     522273.28   
                       FY22                     524802.35   

                                                       \
                                                        
                                                        
                            SALES DISCOUNTS - NA - RW   
Actual   Final Revised FY15                  -2771.56   
                       FY15                   -147.56   
                       FY15                   -278.50   
                       FY15                   -188.61   
                       FY15                   -301.66   
...                                               ...   
Forecast 8+4           FY22                      0.00   
                       FY22                      0.00   
                       FY22                      0.00   
                       FY22                      0.00   
                       FY22                      0.00   

                                                              \
                                                               
                            3010 GERMANY LOHNDATA OPERATIONS   
                                       FEE REVENUE - NA - RW   
Actual   Final Revised FY15                       4638931.53   
                       FY15                        403015.84   
                       FY15             

In [34]:
df[df['period']==0]\
    .loc[("Forecast","6+6","FY22"),
         ("Total Activity", "PR PROC", "Total","Total")]/1000000

54.453345230000004

In [35]:
df[(df['period']<=3)&(df['period']>=1)]\
    .loc[("Forecast","6+6","FY22"),
         ("Total Activity", "PR PROC", "Total","Total")].sum()/1000000

C:\Users\bruno.gonzalez\Miniconda3\envs\azml\lib\site-packages\pandas\core\indexing.py:873: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


13.45463304

## Online Revenue

In [36]:
df = get_df(blob_service_client, blob_list[11], container= container_name)

In [37]:
df

period             Total Activity  \
                                            Total Online Svcs   
                                                Total Paychex   
                                   Total Service Revenue - RW   
Actual   Final Revised FY15      0                114466739.0   
                       FY15      1                  8581821.0   
                       FY15      2                  9700908.0   
                       FY15      3                  8726405.0   
                       FY15      4                  8945846.0   
...                            ...                        ...   
Forecast 8+4           FY22      8                 17506095.0   
                       FY22      9                 16548145.0   
                       FY22     10                 18121060.0   
                       FY22     11                 16694979.0   
                       FY22     12                 16792778.0   

                                                         \
                            Total HRS HR Online Revenue   
                                          Total Paychex   
                             Total Service Revenue - RW   
Actual   Final Revised FY15                  51822600.0   
                       FY15                   3930005.0   
                       FY15                   4535022.0   
                       FY15                   3869726.0   
                       FY15                   3847567.0   
...                                                 ...   
Forecast 8+4           FY22                   4809976.0   
                       FY22                   4426981.0   
                       FY22                   5077572.0   
                       FY22                   4366180.0   
                       FY22                   4419842.0   

                                                        \
                                     HR Online Revenue   
                                         Total Paychex   
                            Total Service Revenue - RW   
Actual   Final Revised FY15                        0.0   
                       FY15                        0.0   
                       FY15                        0.0   
                       FY15                        0.0   
                       FY15                        0.0   
...                                                ...   
Forecast 8+4           FY22                  1631484.0   
                       FY22                  1541484.0   
                       FY22                  1870916.0   
                       FY22                  1473007.0   
                       FY22                  1424429.0   

                                                        \
                                       MSP/LMS Revenue   
                                         Total Paychex   
                            Total Service Revenue - RW   
Actual   Final Revised FY15                        0.0   
                       FY15                        0.0   
                       FY15                        0.0   
                       FY15                        0.0   
                       FY15                        0.0   
...                                                ...   
Forecast 8+4           FY22                  3178492.0   
                       FY22                  2885497.0   
                       FY22                  3206657.0   
                       FY22                  2893173.0   
                       FY22                  2995413.0   

                                                                               \
                            Total Time in a Box/TLO Revenue Total Online Svcs   
                                              Total Paychex     Total Paychex   
                                 Total Service Revenue - RW       Losses - RW   
Actual   Final Revised FY15                      62644139.0           -2747.0   
                   

In [38]:
df["Total Activity", "Total Online Svcs", "Total","Total"] = df["Total Activity"].sum(axis=1)

In [39]:
df[df['period']==0]\
    .loc[("Forecast","6+6","FY22"),
         ("Total Activity","Total Online Svcs", "Total Paychex", "Total Service Revenue - RW")]/1000000

202.755855

In [40]:
df[(df['period']<=3)&(df['period']>=1)]\
    .loc[("Forecast","6+6","FY22"),
         ("Total Activity", "Total Online Svcs", "Total Paychex","Total Service Revenue - RW")]\
    .sum()/1000000

C:\Users\bruno.gonzalez\Miniconda3\envs\azml\lib\site-packages\pandas\core\indexing.py:873: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


49.760486

## Payroll sure

In [14]:
df = get_df(blob_service_client, blob_list[18], container= container_name)\
    .reset_index(-1)\
    .rename(columns={'level_3':'period'})

#Correct month
df['period'] = df['period']\
    .replace({'\nJun': 1, '\nJul': 2, '\nAug': 3, '\nSep': 4, '\nOct': 5, '\nNov': 6, '\nDec': 7, '\nJan': 8,
              '\nFeb': 9, '\nMar': 10, '\nApr': 11, '\nMay': 12, 'YearTotal':0})\
    .astype('int')

In [15]:
df

period                 Total Activity  \
                                   Total Blended Products Revenue   
                                                 TOTAL OPERATIONS   
                                       Total Service Revenue - RW   
Actual   Final Revised FY15      0                   1.236516e+09   
                       FY15      1                   9.667424e+07   
                       FY15      2                   1.179775e+08   
                       FY15      3                   9.652404e+07   
                       FY15      4                   9.144132e+07   
...                            ...                            ...   
Forecast 8+4           FY22      8                   1.023767e+08   
                       FY22      9                   1.019021e+08   
                       FY22     10                   1.150675e+08   
                       FY22     11                   1.006085e+08   
                       FY22     12                   1.066607e+08   

                                                        \
                                                         
                                              CONS HRS   
                            Total Service Revenue - RW   
Actual   Final Revised FY15                36844499.25   
                       FY15                 2959275.81   
                       FY15                 3642193.13   
                       FY15                 2948438.56   
                       FY15                 2756906.24   
...                                                ...   
Forecast 8+4           FY22                 1493093.88   
                       FY22                 1458381.65   
                       FY22                 1644751.31   
                       FY22                 1406579.87   
                       FY22                 1503605.89   

                                                        \
                                                         
                                 Prior Year Adjustment   
                            Total Service Revenue - RW   
Actual   Final Revised FY15                        0.0   
                       FY15                        0.0   
                       FY15                        0.0   
                       FY15                        0.0   
                       FY15                        0.0   
...                                                ...   
Forecast 8+4           FY22                        0.0   
                       FY22                        0.0   
                       FY22                        0.0   
                       FY22                        0.0   
                       FY22                        0.0   

                                                        \
                                Total Delivery Revenue   
                                      TOTAL OPERATIONS   
                            Total Service Revenue - RW   
Actual   Final Revised FY15               1.186875e+08   
                       FY15               7.393352e+06   
                       FY15               1.351374e+07   
                       FY15               7.286967e+06   
                       FY15               7.081862e+06   
...                                                ...   
Forecast 8+4           FY22               2.243552e+07   
                       FY22               6.683674e+06   
                       FY22               7.556914e+06   
                       FY22               1.094369e+07   
                       FY22               7.961770e+06   

                                                        \
                                                         
                                              CONS HRS   
                            Total Service Revenue - RW   
Actual   Final Revised FY15                        0.0   
                       FY15                        0.0   
                       FY15    

In [16]:
df[df['period']==0]\
    .loc[("Forecast","6+6","FY22"),
         ("Total Activity","Total Blended Products Revenue", "TOTAL OPERATIONS", "Total Service Revenue - RW")]/1000000

1304.85610689

In [17]:
df[df['period']==0]\
    .loc[("Forecast","6+6","FY22"),:].groupby(level=1).sum()/1000000

                                            0.000000
HR Solutions (excl PEO)                   598.036988
SurePayroll Revenue                        80.258218
Total Blended Products Revenue           1379.399835
Total Delivery Revenue                    117.029362
Total HR Solutions/ASO (Payroll side)     210.493782
Total Other Processing Revenue            115.501437
Total W-2 Revenue                          87.681535
Name: (Forecast, 6+6, FY22), dtype: float64

In [106]:
80.258218+1379.399835+117.029362+210.493782+115.501437+ 87.681535

1990.3641689999997

## Sure Payroll revenue

In [23]:
df = get_df(blob_service_client, blob_list[19], container= container_name)

In [24]:
df

Total Activity  \
                                                    PR PROC   
                                          1501 SP Direct GB   
                                      FEE REVENUE - NA - RW   
Actual   Final Revised FY15 YearTotal           21219332.78   
                            \nJun                1716994.82   
                            \nJul                2088717.79   
                            \nAug                1411140.00   
                            \nSep                1729748.29   
...                                                     ...   
Forecast 8+4           FY22 \nJan                -154767.08   
                            \nFeb                      0.00   
                            \nMar                      0.00   
                            \nApr                      0.00   
                            \nMay                      0.00   

                                                               \
                                                                
                                                                
                                      OTHER REVENUE - NA - RW   
Actual   Final Revised FY15 YearTotal                    0.00   
                            \nJun                        0.00   
                            \nJul                        0.00   
                            \nAug                        0.00   
                            \nSep                        0.00   
...                                                       ...   
Forecast 8+4           FY22 \nJan                    -2397.90   
                            \nFeb                  -159107.24   
                            \nMar                  -160318.79   
                            \nApr                  -162333.29   
                            \nMay                  -164148.24   

                                                                     \
                                                                      
                                                                      
                                      MAINTENANCE REVENUE - NA - RW   
Actual   Final Revised FY15 YearTotal                          0.00   
                            \nJun                              0.00   
                            \nJul                              0.00   
                            \nAug                              0.00   
                            \nSep                              0.00   
...                                                             ...   
Forecast 8+4           FY22 \nJan                         -14888.45   
                            \nFeb                              0.00   
                            \nMar                              0.00   
                            \nApr                              0.00   
                            \nMay                              0.00   

                                                                 \
                                                                  
                                                                  
                                      SALES DISCOUNTS - NA - RW   
Actual   Final Revised FY15 YearTotal                       0.0   
                            \nJun                           0.0   
                            \nJul                           0.0   
                            \nAug                           0.0   
                            \nSep                           0.0   
...                                                         ...   
Forecast 8+4           FY22 \nJan                     -116200.4   
                            \nFeb                           0.0   
                            \nMar                           0.0   
                            \nApr                           0.0   
                            \nMay                           0.0   

                                                   

In [ ]:
df[df['period']==0]\
    .loc[("Forecast","6+6","FY22"),:].groupby(level=1).sum()/1000000

In [51]:
df["Total Activity", "PR PROC", "Total","Total"] = df["Total Activity"].sum(axis=1)

In [52]:
df[df['period']==0]\
    .loc[("Forecast","6+6","FY22"),
         ("Total Activity","PR PROC", "Total", "Total")]/1000000

97.27571197

## W2

In [53]:
df = get_df(blob_service_client, blob_list[21], container= container_name)

In [54]:
df

period         Total Activity  \
                                   Total Flex W-2 Revenue   
                                               Auburn RSC   
                                    FEE REVENUE - NA - RW   
Actual   Final Revised FY15      0             6549868.12   
                       FY15      1               12851.65   
                       FY15      2               17554.04   
                       FY15      3               11192.36   
                       FY15      4               16141.68   
...                            ...                    ...   
Forecast 8+4           FY22      8                   0.00   
                       FY22      9                   0.00   
                       FY22     10                   0.00   
                       FY22     11                   0.00   
                       FY22     12                   0.00   

                                                           \
                                                            
                                                            
                            MAINTENANCE REVENUE - NA - RW   
Actual   Final Revised FY15                          0.00   
                       FY15                          0.00   
                       FY15                          0.00   
                       FY15                          0.00   
                       FY15                          0.00   
...                                                   ...   
Forecast 8+4           FY22                    4063160.93   
                       FY22                      -2389.86   
                       FY22                       3515.14   
                       FY22                       6520.14   
                       FY22                       6144.04   

                                                                             \
                                                                              
                                                              Rochester RSC   
                            SALES DISCOUNTS - NA - RW FEE REVENUE - NA - RW   
Actual   Final Revised FY15                -189076.13            5504479.81   
                       FY15                      0.00              13090.92   
                       FY15                     19.90              14419.52   
                       FY15                      0.00               6470.22   
                       FY15                      0.00              14781.63   
...                                               ...                   ...   
Forecast 8+4           FY22                -120148.25                  0.00   
                       FY22                      0.00                  0.00   
                       FY22                      0.00                  0.00   
                       FY22                      0.00                  0.00   
                       FY22                      0.00                  0.00   

                                                           \
                                                            
                                                            
                            MAINTENANCE REVENUE - NA - RW   
Actual   Final Revised FY15                          0.00   
                       FY15                          0.00   
                       FY15                          0.00   
                       FY15                          0.00   
                       FY15                          0.00   
...                                                   ...   
Forecast 8+4           FY22                    4095525.80   
                       FY22                       3674.45   
                       FY22                     468983.32   
                       FY22                       5243.23   
                       FY22                       6648.18   

                                                                             \
               

In [55]:
df[df['period']==0]\
    .loc[("Forecast","6+6","FY22"),:].groupby(level=1).sum()/1000000

                                      0.000000
167 Miscellaneous Risk Revenue        0.555490
AFTER THE FACT                        0.004030
AMENDED RETURN PROCESSING            44.181481
BUSINESS TAX SERVICE                  0.120239
ENVELOPES 100                         0.007876
ENVELOPES 200                         0.000185
FULFILLMENT                           0.001272
Flex Readychex Void/Stop Pmt          3.029241
INSTALLATION FEES                     0.001550
LATE FEES 100                         1.171307
LATE FEES 200                         0.058537
MMS Amended Return Processing         0.173673
MMS Miscellaneous Risk Revenue        0.013464
Manual Invoice Pymt Fee               0.290517
NSF CHARGE 100                        3.617537
NSF CHARGE 200                        0.005497
OTHER-PR                             31.007248
OTHER-PREVIEW                         0.607820
PACE LATE FEES                        0.002905
PACE OTHER PR PROC                    0.504947
PAYCHEX ONLIN

In [56]:
df["Total Activity", "Total", "Total","Total"] = df["Total Activity"].sum(axis=1)

In [57]:
df[df['period']==0]\
    .loc[("Forecast","6+6","FY22"),
         ("Total Activity","Total", "Total", "Total")]/1000000

301.09026759

## IFHC

In [58]:
blob_list[6]

'IFHC.txt'

In [59]:
df = get_df(blob_service_client, blob_list[6], container= container_name)

In [61]:
df

period              Total Activity  \
                                                 Total Product   
                                                 Total Paychex   
                                   Interest on Funds Held - RW   
Actual   Final Revised FY15      0                  42081141.0   
                       FY15      1                   3308153.0   
                       FY15      2                   3466374.0   
                       FY15      3                   3434418.0   
                       FY15      4                   3419918.0   
...                            ...                         ...   
Forecast 8+4           FY22      8                   4865483.0   
                       FY22      9                   4676956.0   
                       FY22     10                   4725496.0   
                       FY22     11                   4763628.0   
                       FY22     12                   4585198.0   

                                                         \
                                             TAXPAY 100   
                                      2100 TAXPAY ADMIN   
                            S/T INVESTMENT REVENUE - RW   
Actual   Final Revised FY15                         0.0   
                       FY15                         0.0   
                       FY15                         0.0   
                       FY15                         0.0   
                       FY15                         0.0   
...                                                 ...   
Forecast 8+4           FY22                         0.0   
                       FY22                         0.0   
                       FY22                         0.0   
                       FY22                         0.0   
                       FY22                         0.0   

                                                          \
                              SALES TAX PAYMENT SERVICES   
                            5730 TAX CREDIT SERVICES OPS   
                             S/T INVESTMENT REVENUE - RW   
Actual   Final Revised FY15                          0.0   
                       FY15                          0.0   
                       FY15                          0.0   
                       FY15                          0.0   
                       FY15                          0.0   
...                                                  ...   
Forecast 8+4           FY22                          0.0   
                       FY22                          0.0   
                       FY22                          0.0   
                       FY22                          0.0   
                       FY22                          0.0   

                                                          \
                                              Product NA   
                            5730 TAX CREDIT SERVICES OPS   
                             S/T INVESTMENT REVENUE - RW   
Actual   Final Revised FY15                          0.0   
                       FY15                          0.0   
                       FY15                          0.0   
                       FY15                          0.0   
                       FY15                          0.0   
...                                                  ...   
Forecast 8+4           FY22                          0.0   
                       FY22                          0.0   
                       FY22                          0.0   
                       FY22                          0.0   
                       FY22                          0.0   

                                                          \
                                                 HRS DIV   
                            5730 TAX CREDIT SERVICES OPS   
                             S/T INVESTMENT REVENUE - RW   
Actual   Final Revised FY15                          0.0   
                       FY15                      

In [60]:
df[df['period']==0]\
    .loc[("Forecast","6+6","FY22"),:].sum()/1000000

119.227271

In [63]:
df[df['period']==0]\
    .loc[("Forecast","6+6","FY22"),:].groupby(level=1).sum()/1000000

                               0.000000
401(K) PLANS                   0.210505
DIRECT DEP 100                 1.863276
ESR                            0.003932
EXPENSE MANAGER                0.000074
GARNISHMENTS                   0.004464
HRS DIV                        0.761659
OTHER H&B REVENUE              0.000016
PACE PAYROLL PROC              0.079849
PEO ADMIN Revenue              0.003463
PRD199                         0.010918
Product NA                     0.000000
READYCHEX 100                  0.468613
SALES TAX PAYMENT SERVICES     0.000000
TAXPAY 100                    56.195300
Total Product                 59.613636
WORK COMP                      0.011566
Name: (Forecast, 6+6, FY22), dtype: float64

## Insurance

In [68]:
blob_list[7]

'InsuranceRevenue.txt'

In [69]:
df = get_df(blob_service_client, blob_list[7], container= container_name)

In [70]:
df

period             Total Activity  \
                                       Total Insurance Agency   
                                                Total Paychex   
                                   Total Service Revenue - RW   
Actual   Final Revised FY15      0                131930105.0   
                       FY15      1                 10084069.0   
                       FY15      2                 10353254.0   
                       FY15      3                 10968738.0   
                       FY15      4                 11250149.0   
...                            ...                        ...   
Forecast 8+4           FY22      8                 15110214.0   
                       FY22      9                 15595209.0   
                       FY22     10                 18806565.0   
                       FY22     11                 16717962.0   
                       FY22     12                 16397562.0   

                                                        \
                            Total Workers Comp Revenue   
                                         Total Paychex   
                            Total Service Revenue - RW   
Actual   Final Revised FY15                 64157160.0   
                       FY15                  4858397.0   
                       FY15                  5244319.0   
                       FY15                  5588578.0   
                       FY15                  4968759.0   
...                                                ...   
Forecast 8+4           FY22                  5645056.0   
                       FY22                  5598828.0   
                       FY22                  7568998.0   
                       FY22                  6578746.0   
                       FY22                  6376086.0   

                                                             
                            Total Insurance Service Revenue  
                                              Total Paychex  
                                 Total Service Revenue - RW  
Actual   Final Revised FY15                      67772945.0  
                       FY15                       5225672.0  
                       FY15                       5108935.0  
                       FY15                       5380160.0  
                       FY15                       6281391.0  
...                                                     ...  
Forecast 8+4           FY22                       9465158.0  
                       FY22                       9996381.0  
                       FY22                      11237568.0  
                       FY22                      10139217.0  
                       FY22                      10021477.0  

[182 rows x 4 columns]

In [73]:
df[df['period']==0]\
    .loc[("Forecast","6+6","FY22"),:].groupby(level=1).sum()/1000000

                                     0.000000
Total Insurance Agency             195.034700
Total Insurance Service Revenue    116.443142
Total Workers Comp Revenue          78.591558
Name: (Forecast, 6+6, FY22), dtype: float64

## Other Mgmt

In [75]:
blob_list[13]

'OtherMgmtRevenue.txt'

In [11]:
df = get_df(blob_service_client, blob_list[13], container= container_name)\
    .reset_index(-1)\
    .rename(columns={'level_3':'period'})

#Correct month
df['period'] = df['period'].\
    replace({'\nJun': 1, '\nJul': 2, '\nAug': 3, '\nSep': 4, '\nOct': 5, '\nNov': 6, '\nDec': 7, '\nJan': 8,
             '\nFeb': 9, '\nMar': 10, '\nApr': 11, '\nMay': 12, 'YearTotal':0})\
    .astype('int')

In [12]:
df

period                    Total Activity  \
                                   Other Managment Solutions Revenue   
                                                       Total Paychex   
                                          Total Service Revenue - RW   
Actual   Final Revised FY15      0                        87058529.0   
                       FY15      1                         6350954.0   
                       FY15      2                         7011081.0   
                       FY15      3                         6765242.0   
                       FY15      4                         6369484.0   
...                            ...                               ...   
Forecast 8+4           FY22      8                        23202372.0   
                       FY22      9                        29695313.0   
                       FY22     10                        23337327.0   
                       FY22     11                        21880484.0   
                       FY22     12                        19612352.0   

                                                                    \
                            Total Adv Partner Funding Only Revenue   
                                                     Total Paychex   
                                        Total Service Revenue - RW   
Actual   Final Revised FY15                                    0.0   
                       FY15                                    0.0   
                       FY15                                    0.0   
                       FY15                                    0.0   
                       FY15                                    0.0   
...                                                            ...   
Forecast 8+4           FY22                              6906691.0   
                       FY22                              6370178.0   
                       FY22                              6501591.0   
                       FY22                              6413892.0   
                       FY22                              6666851.0   

                                                                  \
                            Total Unemployment Insurance Revenue   
                                                   Total Paychex   
                                      Total Service Revenue - RW   
Actual   Final Revised FY15                           26268734.0   
                       FY15                            1869396.0   
                       FY15                            2341533.0   
                       FY15                            1986835.0   
                       FY15                            1888067.0   
...                                                          ...   
Forecast 8+4           FY22                            3503156.0   
                       FY22                            3256254.0   
                       FY22                            3726698.0   
                       FY22                            3272264.0   
                       FY22                            3537170.0   

                                                        \
                                        OASIS STAFFING   
                                         Total Paychex   
                            Total Service Revenue - RW   
Actual   Final Revised FY15                        0.0   
                       FY15                        0.0   
                       FY15                        0.0   
                       FY15                        0.0   
                       FY15                        0.0   
...                                                ...   
Forecast 8+4           FY22                      600.0   
                       FY22                        0.0   
                       FY22                        0.0   
                       FY22                        0.0   
                       FY22                        0.0   

    

In [13]:
df[df['period']==0]\
    .loc[("Forecast","6+6","FY22"),:].groupby(level=1).sum()/1000000

                                            0.000000
ESR                                        51.047983
OASIS STAFFING                              0.018555
Other Managment Solutions Revenue         266.228645
Total Adv Partner Funding Only Revenue     77.181851
Total Benetrac Revenue                     18.375835
Total Cafeteria Revenue                    20.037876
Total HRS Other                            56.803986
Total Unemployment Insurance Revenue       42.807407
Name: (Forecast, 6+6, FY22), dtype: float64

In [15]:
df[df['period']==0]\
    .loc[("Forecast","6+6","FY22"),
         ("Total Activity","Other Managment Solutions Revenue", "Total Paychex", "Total Service Revenue - RW")]/1000000

266.228645

## PEO

In [22]:
df = get_df(blob_service_client, blob_list[15], container= container_name)\
    .reset_index(-1)\
    .rename(columns={'level_3':'period'})

#Correct month
df['period'] = df['period'].\
    replace({'\nJun': 1, '\nJul': 2, '\nAug': 3, '\nSep': 4, '\nOct': 5, '\nNov': 6, '\nDec': 7, '\nJan': 8,
             '\nFeb': 9, '\nMar': 10, '\nApr': 11, '\nMay': 12, 'YearTotal':0})\
    .astype('int')

In [23]:
df

period             Total Activity  \
                                            PEO ADMIN Revenue   
                                                Total Paychex   
                                   Total Service Revenue - RW   
Actual   Final Revised FY15      0                 58962873.0   
                       FY15      1                  4472815.0   
                       FY15      2                  4865205.0   
                       FY15      3                  4448330.0   
                       FY15      4                  4705574.0   
...                            ...                        ...   
Forecast 8+4           FY22      8                 25588558.0   
                       FY22      9                 24347449.0   
                       FY22     10                 26993786.0   
                       FY22     11                 26450858.0   
                       FY22     12                 25829451.0   

                                                        \
                                          WORKERS COMP   
                                         Total Paychex   
                            Total Service Revenue - RW   
Actual   Final Revised FY15                 37822568.0   
                       FY15                  2911611.0   
                       FY15                  3091176.0   
                       FY15                  2892685.0   
                       FY15                  2989057.0   
...                                                ...   
Forecast 8+4           FY22                 13686603.0   
                       FY22                 13569988.0   
                       FY22                 14747841.0   
                       FY22                 14811244.0   
                       FY22                 14387945.0   

                                                        \
                                        BENEFITS ADMIN   
                                         Total Paychex   
                            Total Service Revenue - RW   
Actual   Final Revised FY15                 77887776.0   
                       FY15                  4998522.0   
                       FY15                  6244498.0   
                       FY15                  6581757.0   
                       FY15                  6202454.0   
...                                                ...   
Forecast 8+4           FY22                  1825411.0   
                       FY22                  1457815.0   
                       FY22                  1464815.0   
                       FY22                  1471815.0   
                       FY22                  1478815.0   

                                                        \
                                PEO MPP BENEFITS ADMIN   
                                         Total Paychex   
                            Total Service Revenue - RW   
Actual   Final Revised FY15                        0.0   
                       FY15                        0.0   
                       FY15                        0.0   
                       FY15                        0.0   
                       FY15                        0.0   
...                                                ...   
Forecast 8+4           FY22                 24647768.0   
                       FY22                 22783369.0   
                       FY22                 26524634.0   
                       FY22                 21849831.0   
                       FY22                 23353813.0   

                                                  NaN  \
                            Total Health and Benefits   
                                                  NaN   
                                                  NaN   
Actual   Final Revised FY15                77887776.0   
                       FY15                 4998522.0   
                       FY15                 6244498.0   
                       FY15             

In [24]:
df[df['period']==0]\
    .loc[("Forecast","6+6","FY22"),:].groupby(level=1).sum()/1000000

                                0.000000
BENEFITS ADMIN                 17.052661
Other PEO Revenue              29.192781
PEO ADMIN Revenue             309.363048
PEO MPP BENEFITS ADMIN        271.223381
Total Health and Benefits     288.276042
Total PBS Revenue            1287.444434
UNEMPLOYMENT INS 500           88.642904
WORKERS COMP                  175.309461
Name: (Forecast, 6+6, FY22), dtype: float64

In [13]:
df[df['period']==0]\
    .loc[("Forecast","6+6","FY22"),
         ("Total Activity","Total PBS Revenue", "Total Paychex", "Total Service Revenue - RW")]/1000000

890.298017

## Join tables

In [9]:
df_401k = get_df(blob_service_client, blob_list[0], container= container_name)

In [10]:
df_401k.loc[:, ("Total Activity","Total 401k Revenue", "Total Paychex", "Total Service Revenue - RW")]

Actual    Final Revised  FY15  YearTotal    200963139.0
                               \nJun         14883414.0
                               \nJul         15071771.0
                               \nAug         14783933.0
                               \nSep         17075301.0
                                               ...     
Forecast  8+4            FY22  \nJan         25826261.0
                               \nFeb         24740442.0
                               \nMar         25937318.0
                               \nApr         25600810.0
                               \nMay         26080025.0
Name: (Total Activity, Total 401k Revenue, Total Paychex, Total Service Revenue - RW), Length: 182, dtype: float64

In [11]:
df_blen = get_df(blob_service_client, blob_list[4], container= container_name)

In [12]:
df_blen["Total Activity"].sum(axis=1)

Actual    Final Revised  FY15  YearTotal    1.273360e+09
                               \nJun        9.963351e+07
                               \nJul        1.216197e+08
                               \nAug        9.947248e+07
                               \nSep        9.419823e+07
                                                ...     
Forecast  8+4            FY22  \nJan        1.038698e+08
                               \nFeb        1.033605e+08
                               \nMar        1.167122e+08
                               \nApr        1.020151e+08
                               \nMay        1.081643e+08
Length: 182, dtype: float64

In [13]:
df_int = get_df(blob_service_client, blob_list[9], container= container_name)

In [14]:
df_int["Total Activity"].sum(axis=1)

Actual    Final Revised  FY15  YearTotal    9498516.10
                               \nJun         808003.64
                               \nJul         855305.18
                               \nAug         801651.38
                               \nSep         834334.16
                                               ...    
Forecast  8+4            FY22  \nJan        5022326.69
                               \nFeb        4508991.80
                               \nMar        4532101.45
                               \nApr        4385659.84
                               \nMay        4466756.09
Length: 182, dtype: float64

In [15]:
df_online = paychex_ml.clean_data.get_df(blob_service_client, blob_list[11], container= container_name)

In [16]:
df_online.loc[:, ("Total Activity","Total Online Svcs", "Total Paychex", "Total Service Revenue - RW")]

Actual    Final Revised  FY15  YearTotal    114466739.0
                               \nJun          8581821.0
                               \nJul          9700908.0
                               \nAug          8726405.0
                               \nSep          8945846.0
                                               ...     
Forecast  8+4            FY22  \nJan         17506095.0
                               \nFeb         16548145.0
                               \nMar         18121060.0
                               \nApr         16694979.0
                               \nMay         16792778.0
Name: (Total Activity, Total Online Svcs, Total Paychex, Total Service Revenue - RW), Length: 182, dtype: float64

In [20]:
df_sure = get_df(blob_service_client, blob_list[19], container= container_name)

In [25]:
df_sure = df_sure.reset_index(-1)\
    .rename(columns={'level_3':'period'})

#Correct month
df_sure['period'] = df_sure['period']\
    .replace({'\nJun': 1, '\nJul': 2, '\nAug': 3, '\nSep': 4, '\nOct': 5, '\nNov': 6, '\nDec': 7, '\nJan': 8,
             '\nFeb': 9, '\nMar': 10, '\nApr': 11, '\nMay': 12, 'YearTotal':0})\
    .astype('int')

In [30]:
blob_list[18]

'PayrollSurePayrollASOInternationalHighLevelRevenue.txt'

In [21]:
df_pay = get_df(blob_service_client, blob_list[18], container= container_name)
# df_pay = df_pay.reset_index(-1)\
#     .rename(columns={'level_3':'period'})
#
# #Correct month
# df_pay['period'] = df_pay['period']\
#     .replace({'\nJun': 1, '\nJul': 2, '\nAug': 3, '\nSep': 4, '\nOct': 5, '\nNov': 6, '\nDec': 7, '\nJan': 8,
#               '\nFeb': 9, '\nMar': 10, '\nApr': 11, '\nMay': 12, 'YearTotal':0})\
#     .astype('int')

In [22]:
df_pay

Total Activity  \
                                      Total Blended Products Revenue   
                                                    TOTAL OPERATIONS   
                                          Total Service Revenue - RW   
Actual   Final Revised FY15 YearTotal                   1.236516e+09   
                            \nJun                       9.667424e+07   
                            \nJul                       1.179775e+08   
                            \nAug                       9.652404e+07   
                            \nSep                       9.144132e+07   
...                                                              ...   
Forecast 8+4           FY22 \nJan                       1.023767e+08   
                            \nFeb                       1.019021e+08   
                            \nMar                       1.150675e+08   
                            \nApr                       1.006085e+08   
                            \nMay                       1.066607e+08   

                                                                  \
                                                                   
                                                        CONS HRS   
                                      Total Service Revenue - RW   
Actual   Final Revised FY15 YearTotal                36844499.25   
                            \nJun                     2959275.81   
                            \nJul                     3642193.13   
                            \nAug                     2948438.56   
                            \nSep                     2756906.24   
...                                                          ...   
Forecast 8+4           FY22 \nJan                     1493093.88   
                            \nFeb                     1458381.65   
                            \nMar                     1644751.31   
                            \nApr                     1406579.87   
                            \nMay                     1503605.89   

                                                                  \
                                                                   
                                           Prior Year Adjustment   
                                      Total Service Revenue - RW   
Actual   Final Revised FY15 YearTotal                        0.0   
                            \nJun                            0.0   
                            \nJul                            0.0   
                            \nAug                            0.0   
                            \nSep                            0.0   
...                                                          ...   
Forecast 8+4           FY22 \nJan                            0.0   
                            \nFeb                            0.0   
                            \nMar                            0.0   
                            \nApr                            0.0   
                            \nMay                            0.0   

                                                                  \
                                          Total Delivery Revenue   
                                                TOTAL OPERATIONS   
                                      Total Service Revenue - RW   
Actual   Final Revised FY15 YearTotal               1.186875e+08   
                            \nJun                   7.393352e+06   
                            \nJul                   1.351374e+07   
                            \nAug                   7.286967e+06   
                            \nSep                   7.081862e+06   
...                                                          ...   
Forecast 8+4           FY22 \nJan                   2.243552e+07   
                            \nFeb                   6.683674e+06   
                            \nMar                   7.556914e+06   
                            \nApr              

In [29]:
df_pay.groupby(level=1, axis=1).sum()

HR Solutions (excl PEO)  \
Actual   Final Revised FY15 YearTotal              280104498.0   
                            \nJun                   21143605.0   
                            \nJul                   25838048.0   
                            \nAug                   21762760.0   
                            \nSep                   20288451.0   
...                                                        ...   
Forecast 8+4           FY22 \nJan                   49289702.0   
                            \nFeb                   47754765.0   
                            \nMar                   55976261.0   
                            \nApr                   49129952.0   
                            \nMay                   52095246.0   

                                       SurePayroll Revenue  \
Actual   Final Revised FY15 YearTotal                 0.00   
                            \nJun                     0.00   
                            \nJul                     0.00   
                            \nAug                     0.00   
                            \nSep                     0.00   
...                                                    ...   
Forecast 8+4           FY22 \nJan               7489372.86   
                            \nFeb               7565854.96   
                            \nMar               7557327.00   
                            \nApr               7649738.88   
                            \nMay               7685813.37   

                                       Total Blended Products Revenue  \
Actual   Final Revised FY15 YearTotal                    1.319551e+09   
                            \nJun                        1.033067e+08   
                            \nJul                        1.254030e+08   
                            \nAug                        1.032005e+08   
                            \nSep                        9.802494e+07   
...                                                               ...   
Forecast 8+4           FY22 \nJan                        1.077344e+08   
                            \nFeb                        1.069236e+08   
                            \nMar                        1.202123e+08   
                            \nApr                        1.053951e+08   
                            \nMay                        1.116110e+08   

                                       Total Delivery Revenue  \
Actual   Final Revised FY15 YearTotal            1.186875e+08   
                            \nJun                7.393352e+06   
                            \nJul                1.351374e+07   
                            \nAug                7.286967e+06   
                            \nSep                7.081862e+06   
...                                                       ...   
Forecast 8+4           FY22 \nJan                2.243552e+07   
                            \nFeb                6.683674e+06   
                            \nMar                7.556914e+06   
                            \nApr                1.094369e+07   
                            \nMay                7.961770e+06   

                                       Total HR Solutions/ASO (Payroll side)  \
Actual   Final Revised FY15 YearTotal                            96688929.75   
                            \nJun                                 7255027.55   
                            \nJul                                 9070820.80   
                            \nAug                                 7489159.50   
                            \nSep                                 7011292.24   
...                                                                      ...   
Forecast 8+4           FY22 \nJan                                17242037.23   
                            \nFeb                                16746239.66   
                            \nMar                                19830263.01   
                            \nApr          

### Join function

In [8]:
files = [blob_list[i] for i in [0,4,9,11,19,7,15,13,18]]

In [9]:
# column_names = ['20 Total 401k',
#                 '11 Payroll Blended Products',
#                 '17 Total International',
#                 '40 Total Online Services',
#                 '16 SurePayroll',
#                 '70 Total Insurance Services',
#                 '60 Total PEO',
#                 '50 Other Managment Solutions',
#                 '31 HR Solutions (excl PEO)',
#                 'pop1',
#                 'pop2',
#                 '13 Delivery Revenue',
#                 '14 ASO Allocation',
#                 '15 Other Processing Revenue',
#                 '12 W2 Revenue',
#                 ]

In [10]:
column_names = {
    ('Total Activity', 'Total 401k Revenue', 'Total Paychex', 'Total Service Revenue - RW'): '20 Total 401k',
     0: '11 Payroll Blended Products',
     1: '17 Total International',
     ('Total Activity', 'Total Online Svcs', 'Total Paychex', 'Total Service Revenue - RW'): '40 Total Online Services',
     2: '16 SurePayroll',
     ('Total Activity', 'Total Insurance Agency', 'Total Paychex', 'Total Service Revenue - RW'): '70 Total Insurance Services',
     ('Total Activity', 'Total PBS Revenue', 'Total Paychex', 'Total Service Revenue - RW'): '60 Total PEO',
     ('Total Activity', 'Other Managment Solutions Revenue', 'Total Paychex', 'Total Service Revenue - RW'): '50 Other Managment Solutions',
     'HR Solutions (excl PEO)': '31 HR Solutions (excl PEO)',
     'SurePayroll Revenue': 'pop1',
     'Total Blended Products Revenue': 'pop2',
     'Total Delivery Revenue': '13 Delivery Revenue',
     'Total HR Solutions/ASO (Payroll side)': '14 ASO Allocation',
     'Total Other Processing Revenue': '15 Other Processing Revenue',
     'Total W-2 Revenue': '12 W2 Revenue'
}

In [11]:
df = join_all(blob_service_client, files, column_names)

401kRevenue.txt
BlendedProductRevenue.txt
InternationalRevenue.txt
OnlineRevenue.txt
SurePayollRevenue.txt
InsuranceRevenue.txt
PEORevenue.txt
OtherMgmtRevenue.txt
PayrollSurePayrollASOInternationalHighLevelRevenue.txt


In [12]:
df[df['period']==0]\
    .loc[("Forecast","6+6","FY22"),:].groupby(level=0).sum()/1000000

11 Payroll Blended Products     1327.136744
12 W2 Revenue                     87.681535
13 Delivery Revenue              117.029362
14 ASO Allocation                210.493782
15 Other Processing Revenue      115.501437
16 SurePayroll                    97.275712
17 Total International            54.453345
20 Total 401k                    306.228870
31 HR Solutions (excl PEO)       598.036988
40 Total Online Services         202.755855
50 Other Managment Solutions     266.228645
60 Total PEO                     890.298017
70 Total Insurance Services      195.034700
period                             0.000000
Name: (Forecast, 6+6, FY22), dtype: float64